# Assignment 1

Modify the regression scratch code in our lecture such that:

- Implement early stopping in which if the absolute difference between old loss and new loss does not exceed certain threshold, we abort the learning.

- Implement options for stochastic gradient descent in which we use only one sample for training. Make sure that sample does not repeat unless all samples are read at least once already.

- Put everything into class.

In [46]:
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
from time import time
import random

#1.1 Get X and y in the right shape
boston = load_boston()
X = boston.data
m = X.shape[0]    #number of samples
n = X.shape[1]    #number of features
y = boston.target
assert m == y.shape[0]     #number of rows in X is the same as number of rows in y


#1.2 Feature scale my data to reach faster convergence
scaler = StandardScaler()
X = scaler.fit_transform(X)     #Fit to data then transform it


#1.3 Train test split my data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
assert len(X_train) == len(y_train)
assert len(X_test) == len(y_test)

#1.4 Add intercepts
#w0 is my intercept
#np.ones((shape))
intercept = np.ones((X_train.shape[0], 1))

#concatenate the intercept based on axis=1
X_train = np.concatenate((intercept, X_train), axis=1)

#np.ones((shape))
intercept = np.ones((X_test.shape[0], 1))

#concatenate the intercept based on axis=1
X_test = np.concatenate((intercept, X_test), axis=1)


class LinearRegression:
    
    #select method: 'batch', 'sgd', 'mini_batch'
    def __init__(self, alpha = 0.001, max_iter = 1000, tol = 0.0001, iter_stop = 10**5, method = 'batch', n_sample = 100):
   
        self.alpha = alpha
        self.max_iter = max_iter
        self.tol = tol
        self.iter_stop = iter_stop
        self.method = method
        self.n_sample = n_sample
        
    def h_theta(self, X, theta):
        return X @ theta

    def mse(self, yhat, y):
        return ((yhat - y)**2).sum() / np.shape(yhat)[0]

    def mse_sgd(self, yhat, y):
        return (yhat - y)**2

    def gradient(self, X, error):
        return X.T @ error

    def gradient_sgd(self, X, error):
        return np.array(np.matrix(list(X)).T @ [error]).reshape(-1)
    
    def predict(self, X_test):
        yhat = self.h_theta(X_test, self.theta)
        return yhat
        
    def fit(self, X_train, y_train):

        start = time()
        assert X_train.shape[0] == y_train.shape[0]
               
        self.theta = np.zeros(X_train.shape[1])
        self.mse_hist_train = []      #without replacement
        random_list_train = list(np.arange(0, len(X_train)))    
         
        if self.method == 'batch':
            for i in range(self.max_iter):
                yhat = self.h_theta(X_train, self.theta)
                error = yhat - y_train
                grad = self.gradient(X_train, error)
                self.theta = self.theta - self.alpha * grad

                # MSE of Training
                mse_train = self.mse(yhat, y_train)
                self.mse_hist_train.append(mse_train)

                # Check condition
                # Condition 1: if loss_diff < tolerance (consider -> training set)
                if i == 0:
                    pass
                else:
                    loss_diff = np.abs(self.mse_hist_train[-1] - self.mse_hist_train[-2])
                    if (loss_diff <= self.tol):
                        break

                # Condition 2: stop in the iteration that we want
                if i == self.iter_stop:
                    break
    
    
        elif self.method == 'sdg':
            for i in range(self.max_iter):
                for j in random_list_train:
                    # Random data : Stochastic Gradient Descent
                    random_index_train = random.choice(random_list_train)
                    random_list_train.remove(random_index_train)

                    # Reset random_list again if no element in the list anymore
                    if len(random_list_train) == 0:
                        random_list_train = list(np.arange(0, len(X_train)))

                    yhat = self.h_theta(X_train[random_index_train], self.theta)
                    error = yhat - y_train[random_index_train]
                    grad = self.gradient_sgd(X_train[random_index_train], error)
                    self.theta = self.theta - self.alpha * grad

                # MSE of Training
                mse_train = self.mse_sgd(yhat, y_train[random_index_train])
                self.mse_hist_train.append(mse_train)

                # Check condition
                # Condition 1: if loss_diff < tolerance (condider -> training set)
                if i == 0:
                    pass
                else:
                    loss_diff = np.abs(self.mse_hist_train[-1] - self.mse_hist_train[-2])
                    if (loss_diff <= self.tol):
                        break

                # Condition 2: stop in the iteration that we want
                if i == self.iter_stop-1:
                    break
                    
                    
        elif self.method == 'mini_batch':
            for i in range(self.max_iter):
                # Random data : Mini-Batch Stochastic Gradient Descent
                if self.n_sample > len(X_train):
                    self.n_sample = len(X_train)

                random_index_batch = random.sample(random_list_train, k = self.n_sample)
                yhat = self.h_theta(X_train[random_index_batch], self.theta)
                error = yhat - y_train[random_index_batch]
                grad = self.gradient(X_train[random_index_batch], error)
                self.theta = self.theta - self.alpha * grad

                # MSE of Training
                mse_train = self.mse(yhat, y_train[random_index_batch])
                self.mse_hist_train.append(mse_train)

                # Check condition
                # Condition 1: if loss_diff < tolerance (condider -> training set)
                if i == 0:
                    pass
                else:
                    loss_diff = np.abs(self.mse_hist_train[-1] - self.mse_hist_train[-2])
                    if (loss_diff <= self.tol):
                        break

                # Condition 2: stop in the iteration that we want
                if i == self.iter_stop:
                    break   
                    
                    
        time_taken = time() - start  
        print("Stop at iteration: ", i+1)
        print("Time used: ", time_taken)
        
        return self
    
    
    

In [47]:
#########################################################
# Early stopping
batch_model = LinearRegression(alpha = 0.0001, max_iter = 1000,  tol = 0.0001, method = 'batch')
batch_model.fit(X_train, y_train)
y_hat = batch_model.predict(X_test)
mse = batch_model.mse(y_hat, y_test)

# print the mse
print("MSE: ", mse)

Stop at iteration:  804
Time used:  0.03491067886352539
MSE:  22.2190616714931


In [48]:
#########################################################
# SDG
sdg_model = LinearRegression(alpha = 0.0001, max_iter = 1000,  tol = 0.0001, method = 'sdg')
sdg_model.fit(X_train, y_train)
y_hat = sdg_model.predict(X_test)
mse = sdg_model.mse(y_hat, y_test)

# print the mse
print("MSE: ", mse)

Stop at iteration:  1000
Time used:  2.3546857833862305
MSE:  22.5103281567019


In [49]:
#########################################################
# Mini-Batch
mini_model = LinearRegression(alpha = 0.0001, max_iter = 1000,  tol = 0.0001, method = 'mini_batch', n_sample = 100)
mini_model.fit(X_train, y_train)
y_hat = mini_model.predict(X_test)
mse = mini_model.mse(y_hat, y_test)

# print the mse
print("MSE: ", mse)

Stop at iteration:  1000
Time used:  0.22979116439819336
MSE:  22.109245739557924
